In [1]:
import pandas as pd
import numpy as np 
import json
import csv


import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm_notebook as tqdm

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('averaged_perceptron_tagger')

import spacy
import math

import string
import sys
import random

from collections import Counter
from itertools import chain

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aarunku5/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
df_wino = pd.read_csv('./test_good_101_1.csv')
nlp = spacy.load("en_trf_bertbaseuncased_lg")
words=pd.DataFrame()
frames=[df_wino['sentence1'],df_wino['sentence2']]
words['full'] = pd.concat(frames)
words.reset_index(inplace = True) 
words['casewords'] = words['full'].str.lower()
words['wsw'] = words['casewords'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
words['nopunc'] = words['wsw'].str.replace("'", "")
words['nopunc'] = words['nopunc'].str.replace(".", "")
words['nopunc'] = words['nopunc'].str.replace(",", "")
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
words['text_lemmatized'] = words.nopunc.apply(lemmatize_text)
words['final']="NaN"
def final(x):
    makeitastring = ' '.join(map(str, x))
    makeitastring=makeitastring.replace(",","")
    makeitastring=makeitastring.replace("'","")
    makeitastring=makeitastring.replace("[","")
    makeitastring=makeitastring.replace("]","")
    return makeitastring
for i in tqdm(range(len(words))):    
    column=['text_lemmatized']
    row=[i]
    words.loc[i,'final']=final(words.loc[i,'text_lemmatized'])
words['fullnopunc'] = words['full'].str.replace("'", "")
words['fullnopunc'] = words['fullnopunc'].str.replace(".", "")
words['fullnopunc'] = words['fullnopunc'].str.replace(",", "")
words['label']='NaN'
for i in tqdm(range(len(df_wino))):
    words.loc[i,'label']=df_wino.loc[i,'gold_label']
    words.loc[(i+len(df_wino)),'label']=df_wino.loc[i,'gold_label']
words.to_csv('./test_good_101_1_words.csv', index = False)
e=pd.DataFrame(columns=['label','final','fullnopunc'])
n=pd.DataFrame(columns=['label','final','fullnopunc'])
c=pd.DataFrame(columns=['label','final','fullnopunc'])
#split test labels
for i in tqdm(range(len(words))):
    l=words.loc[i,'label']
    new_row={'label':words.loc[i,'label'],'final':words.loc[i,'final'],'fullnopunc':words.loc[i,'fullnopunc']}
    if(l=='entailment'):
        e=e.append(new_row, ignore_index=True)
    elif(l=='neutral'):
        n=n.append(new_row, ignore_index=True)
    else:
        c=c.append(new_row,ignore_index=True)
e.to_csv('./test_good_101_1_wordse.csv', index = False)
n.to_csv('./test_good_101_1_wordsn.csv', index = False)
c.to_csv('./test_good_101_1_wordsc.csv', index = False)











In [3]:
print(len(e))
print(len(n))
print(len(c))
print(len(df_wino))
print(len(words))

48
104
50
101
202


In [5]:
gwords=pd.read_csv("./test_good_101_1_words.csv")
size=len(gwords)
WSIML=0.5
sentence=gwords['fullnopunc']
lists=[]
vals=[]
for x in tqdm(range(len(sentence))):
    arr=[]
    s=sentence[x]
    word = s.split()
    length=len(word)
    sl=0
    for i in (range(length)):
        sm=0
        for j in (range(length)):
            if(i!=j):
                sm=sm+nlp(word[i]).similarity(nlp(word[j]))
        sm=sm/length
        arr.append(sm)
        sm=abs(sm-WSIML)
        sl=sl+sm
    lists.append(arr)
    vals.append(size/sl)

len(gwords)/pd.Series(vals).sum()

0.006532405592774371

In [4]:
df_testgood=pd.read_csv("./test_good_101_1.csv")

SIML=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
arr=[]
df = pd.DataFrame(columns=['p', 'h'], data=df_testgood[['sentence1','sentence2']].values)
df['p'] = df['p'].str.replace("'", "")
df['p'] = df['p'].str.replace(".", "")
df['p'] = df['p'].str.replace(",", "")
df['h'] = df['h'].str.replace("'", "")
df['h'] = df['h'].str.replace(".", "")
df['h'] = df['h'].str.replace(",", "")

for x in tqdm(range(len(SIML))):
    dr=0
    for i in tqdm(range(len(df_testgood))):        
        dr=dr+abs(((nlp(df.loc[i,'p'])).similarity(nlp(df.loc[i,'h'])))-SIML[x])
    arr.append(len(df_testgood)/dr)
print(arr)
    
#-------------------------------------------------------------------------------------------------------

garr=[]

df = pd.DataFrame(columns=['p', 'h'], data=df_testgood[['sentence1','sentence2']].values)
df['p'] = df['p'].str.replace("'", "")
df['p'] = df['p'].str.replace(".", "")
df['p'] = df['p'].str.replace(",", "")
df['h'] = df['h'].str.replace("'", "")
df['h'] = df['h'].str.replace(".", "")
df['h'] = df['h'].str.replace(",", "")

for i in tqdm(range(len(df_testgood))):         
    garr.append(abs((len((df.iloc[i]['p']).split()))-(len((df.iloc[i]['h']).split()))))
print(garr)
print(pd.Series(garr).std()/len(df_testgood))    
print(len(garr)/(pd.Series(garr).sum()+1))

#-------------------------------------------------------------------------------------------------------

gsarr=[]
df = pd.DataFrame(columns=['p', 'h'], data=df_testgood[['sentence1','sentence2']].values)
df['p'] = df['p'].str.replace("'", "")
df['p'] = df['p'].str.replace(".", "")
df['p'] = df['p'].str.replace(",", "")
df['h'] = df['h'].str.replace("'", "")
df['h'] = df['h'].str.replace(".", "")
df['h'] = df['h'].str.replace(",", "")

for i in tqdm(range(len(df_testgood))):        
    gsarr.append(abs(((nlp(df.loc[i,'p'])).similarity(nlp(df.loc[i,'h'])))))
print(gsarr)
    
print(pd.Series(gsarr).std()/len(df_testgood))



















[1.5078036884658743, 1.7755167605086704, 2.158818625811296, 2.75318032167654, 3.774922918891222, 5.759273105786393, 9.559867544343893, 11.618984979000448, 7.11004329237453]



[1, 9, 4, 6, 2, 6, 1, 8, 15, 4, 1, 7, 11, 1, 5, 4, 12, 3, 8, 11, 11, 11, 2, 3, 3, 4, 2, 19, 1, 7, 18, 6, 2, 8, 2, 0, 6, 5, 4, 6, 31, 38, 5, 16, 4, 2, 5, 33, 12, 16, 16, 4, 5, 10, 1, 2, 9, 8, 3, 5, 6, 1, 4, 0, 2, 1, 18, 9, 5, 2, 9, 1, 4, 6, 5, 0, 17, 6, 19, 16, 2, 2, 9, 7, 13, 7, 1, 4, 18, 3, 1, 2, 10, 6, 7, 1, 7, 8, 10, 7, 3]
0.06756992585061462
0.13950276243093923



[0.731594078373792, 0.7071323328916573, 0.6084232871713664, 0.83258588081206, 0.8659693743389781, 0.8373076248945541, 0.785089759464352, 0.711765019821002, 0.7204732725577601, 0.7578399374810542, 0.8200743273283415, 0.8207343656307879, 0.7751139141883853, 0.8816582624539723, 0.8842817887763237, 0.9313493167365009, 0.7348446484908576, 0.6534945662526068, 0.673514136586978, 0.6552793109798888, 0.8283842707943571, 0.8180516428091832, 0.7768818276487424, 0.6008899534498305, 0.6728583850337511, 0.726095322785707, 0.8581615607466013, 0.7653848492190759, 0.9160845289055501, 0.8116007957688067, 0.684753683096059, 0.7966191692278501, 0.8166835717099845, 0.8084913075183292, 0.7762492988386241, 0.6970762385609213, 0.5681490553821698, 0.8961349735801857, 0.8999271642928709, 0.5096694766383505, 0.7170249399407199, 0.7647675868650998, 0.8735501884536386, 0.7822725363055315, 0.6545521273466435, 0.7270936806490413, 0.8528961190603768, 0.6776162638926507, 0.5409919781502396, 0.6502575025945189, 0.6241